In [4]:
####A项目：爬取大众品牌汽车在易车网上的报价数据 工号：15785 姓名：彭宇 #####
import requests
from bs4 import BeautifulSoup
import pandas as pd
#得到页面内容
def get_page_content(url):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html = requests.get(url,headers=headers,timeout=10)
    content = html.text
    #通过content创建BS对象
    soup = BeautifulSoup(content,'html.parser')
    return soup

#分析页面所需内容并返回
def analysis(soup):
    #找到具备完整车型及报价的所有div类
    item_list = soup.find_all('div',class_='search-result-list-item')
    #创建DataFrame,对要爬取数据的各字段进行命名
    df = pd.DataFrame(columns = ['名称', '最低价格', '最高价格', '产品图片链接'])
    for item in item_list:
        info = {}#空字典用于存放报价信息
        car_model = item.find('p',class_='cx-name text-hover').text#提取车型名称
        price = item.find('p',class_='cx-price').text#提取车型价格
        if str(price) != '暂无':#判断是否暂无价格
            price_list = price.split('-',1)#以'-'符号将价格分隔为最低与最高价格两个字符串
            price_low = price_list[0] + '万'#最低价格，在价格数字后添加'万'字
            price_high = price_list[1]#最高价格(包含末尾万字)
        else:
            price_high = '暂无'
            price_low = '暂无'
        basic_url='http:'#为图片链接准备前缀
        img_url = basic_url + item.find('img',class_='img')['src']#提取图片链接并加上前缀
        #将各项提取内容赋值到字典中
        info['名称'], info['最低价格'], info['最高价格'], info['产品图片链接'] = \
            car_model, price_low, price_high, img_url
        df = df.append(info,ignore_index=True)#将报价信息添加到Dataframe中
        
    return df#返回Dataframe
#主函数
if __name__=="__main__":
    url = 'http://car.bitauto.com/xuanchegongju/?l=8&mid=8'#易车网大众界面网页链接
    soup = get_page_content(url)#由链接获取页面内容
    result = analysis(soup)#分析页面内容获取报价数据dataframe
    print(result)
    result.to_csv('大众品牌汽车报价数据.csv',index=False,encoding='utf_8_sig')#提取爬取数据保存为csv文件

            名称    最低价格    最高价格  \
0           探岳  18.59万  31.39万   
1           途岳  16.58万  22.38万   
2          途观L  21.88万  31.18万   
3           途昂  29.99万  50.59万   
4           途铠  11.49万  15.99万   
5           探歌  13.58万  20.38万   
6           探影  11.49万  15.99万   
7           途观  19.48万  23.18万   
8          探岳X  23.58万  29.58万   
9          途昂X  28.99万  43.99万   
10          途锐  56.98万  82.18万   
11    途观L 插电混动  26.58万  32.58万   
12  探岳GTE 插电混动  24.98万  25.98万   
13      Tiguan  23.98万  35.08万   
14     途锐 插电混动      暂无      暂无   

                                               产品图片链接  
0   http://img2.bitautoimg.com/autoalbum/files/201...  
1   http://img2.bitautoimg.com/autoalbum/files/201...  
2   http://img4.bitautoimg.com/autoalbum/files/201...  
3   http://img3.bitautoimg.com/autoalbum/files/201...  
4   http://img2.bitautoimg.com/autoalbum/files/201...  
5   http://img4.bitautoimg.com/autoalbum/files/201...  
6   http://img1.bitautoimg.com/autoalbum/files/201...  
7   htt